In [0]:
#Resources: https://www.youtube.com/watch?reload=9&v=a4teH_l3q0c

import pandas as pd
import numpy as np
import torch
from sklearn.metrics import mean_squared_error
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim #This is a package implementing various optimization algorithms

In [61]:
tensor = torch.Tensor([1,2,3])
print(tensor)

numpy_array = np.array([1,2,3])
tensor2 = torch.from_numpy(numpy_array)
print(tensor2)

tensor([1., 2., 3.])
tensor([1, 2, 3])


In [62]:
print(tensor.dot(tensor))
print(tensor2 ** 2)
print(tensor + 2)

tensor(14.)
tensor([1, 4, 9])
tensor([3., 4., 5.])


In [63]:
data = pd.read_csv("sample_data/california_housing_train.csv")
data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
5,-114.58,33.63,29.0,1387.0,236.0,671.0,239.0,3.3438,74000.0
6,-114.58,33.61,25.0,2907.0,680.0,1841.0,633.0,2.6768,82400.0
7,-114.59,34.83,41.0,812.0,168.0,375.0,158.0,1.7083,48500.0
8,-114.59,33.61,34.0,4789.0,1175.0,3134.0,1056.0,2.1782,58400.0
9,-114.60,34.83,46.0,1497.0,309.0,787.0,271.0,2.1908,48100.0


In [0]:
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

In [0]:
class Net(nn.Module): #Our Net class is going to inherit from nn.Module (which is the base class for all neural network modules)
  def __init__(self): #Constructor
    super(Net, self).__init__() #Super keyword makes it possible for us to call the __init__ method from the parent class

    #nn.Linear() is going to multiply each neuron by each weights and add the bias term
    #Fully connected layers
    #Here we are using 2 hidden layers but this is something you need to try and see what works best
    self.fc1 = nn.Linear(in_features=8, out_features=100) #8 input features will be connected to other 100 on the first layer
    self.fc2 = nn.Linear(in_features=100, out_features=100) #This second fully connected layer is going to get the previous 100 units and connect to other 100
    self.output = nn.Linear(in_features=100, out_features=1) #This last layer will have 1 output neuron which is going to represent the house's predicted value

  def forward_propagation(self, x):
    #We are going to use a hidden layer to do the feature scaling process for us in order to make the process faster
    x = F.normalize(x) #x is now our normalized feature array
    x = F.relu(self.fc1(x)) #The Relu activation function will be applied to our first layer 
    x = F.relu(self.fc2(x)) #The Relu activation function will be applied to our second layer 

    y = self.output(x) #This is our output layer with the predictions

    return y  

In [66]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=8, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (output): Linear(in_features=100, out_features=1, bias=True)
)


In [0]:
X_tensor = torch.from_numpy(data.values[:, :-1]).float() 
y_tensor = torch.from_numpy(data.values[:, -1]).float()

#Since our features have already been normalized, we have to do the same with our target 
y_tensor = torch.log(y_tensor)

In [68]:
#net.parameters(): matrices of the weights which connects the neurons of our Neural Network
list(net.parameters())

[Parameter containing:
 tensor([[-0.0933,  0.0148,  0.2695,  0.2668,  0.2264,  0.2147,  0.0171,  0.1570],
         [ 0.2298,  0.1852,  0.2017, -0.2311, -0.1907,  0.1726, -0.0300,  0.2542],
         [-0.2397, -0.0038, -0.2607, -0.2091,  0.0394, -0.3170, -0.1863, -0.2380],
         [ 0.2346,  0.0432, -0.1587, -0.3360, -0.2367,  0.1873, -0.0891,  0.2626],
         [ 0.0476,  0.0260,  0.1953, -0.2039,  0.1932,  0.1339, -0.0006,  0.1404],
         [-0.2298,  0.2012, -0.3045,  0.0469,  0.3361, -0.0681, -0.0759,  0.2930],
         [ 0.2537, -0.0025,  0.2493,  0.2553, -0.0875,  0.1657, -0.1190,  0.1767],
         [-0.3030, -0.2319,  0.1101,  0.3287, -0.2544,  0.3131,  0.2389, -0.3001],
         [ 0.0713,  0.1440, -0.3362, -0.2107,  0.2271,  0.2871, -0.1209, -0.0161],
         [-0.0994, -0.2009, -0.2164,  0.1614,  0.2313, -0.2359, -0.2912, -0.2842],
         [-0.3509, -0.3470,  0.1452, -0.1363,  0.0423,  0.2872, -0.1846,  0.1149],
         [-0.0165,  0.0920,  0.2973, -0.1023, -0.0209, -0.2857, 

In [0]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [0]:
#In order to check whether the optimizer is good, we are going to evaluate it using RMSE 
criterion = nn.MSELoss() #The main goal is to reduce the error

In [84]:
for i in range(100):
  #In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward
  #passes. This is convenient while training RNNs. Because of this, when you start your training loop, ideally you should zero out the gradients so that you 
  #do the parameter update correctly. Else the gradient would point in some other direction than the intended direction towards the minimum.
  optimizer.zero_grad() #Set optimizer gradient functions equal to zero
  prediction = net.forward_propagation(X_tensor) #Here we are using the 'net' object with the tensor containing the data we want to train  
  loss = criterion(prediction, y_tensor)
  #backward() is going to calculate all the gradients so then the optimizer might use them by adjusting the weights in order to minimize our function
  loss.backward()
  optimizer.step() #Performs a parameter update based on the current gradient 
  
  prediction2 = torch.exp(net.forward_propagation(X_tensor))
  loss = criterion(prediction2, torch.exp(y_tensor))
  print('Monitoring the loss: ', torch.sqrt(loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([17000])) that is different to the input size (torch.Size([17000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Monitoring the loss:  tensor(127059.6406, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126999.9297, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126941.6406, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126882.4531, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126825.8984, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126767.9609, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126709.9141, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126653.3750, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126595.8359, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126538.0625, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126481.7188, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126424.3984, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126364.6328, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126307.5781, grad_fn=<SqrtBackward>)
Monitoring the loss:  tensor(126250.5781, grad_fn=<SqrtBackward>)
Monitoring

In [86]:
test_df = pd.read_csv("sample_data/california_housing_test.csv")
test_df.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
5,-119.56,36.51,37.0,1018.0,213.0,663.0,204.0,1.6635,67000.0
6,-121.43,38.63,43.0,1009.0,225.0,604.0,218.0,1.6641,67000.0
7,-120.65,35.48,19.0,2310.0,471.0,1341.0,441.0,3.2250,166900.0
8,-122.84,38.40,15.0,3080.0,617.0,1446.0,599.0,3.6696,194400.0
9,-118.02,34.08,31.0,2402.0,632.0,2830.0,603.0,2.3333,164200.0


In [0]:
X_test_tensor = torch.from_numpy(test_df.values[:, :-1]).float()
y_test = test_df.values[:, -1]

In [90]:
prediction_test = torch.exp(net.forward_propagation(X_test_tensor))
print(prediction_test)

prediction_test_numpy = prediction_test.detach().numpy()

tensor([[160062.9062],
        [188834.1562],
        [148795.7500],
        ...,
        [192273.7812],
        [248282.2031],
        [168334.3750]], grad_fn=<ExpBackward>)


In [91]:
y_test = test_df.values[:, -1]
np.sqrt(mean_squared_error(y_test, prediction_test_numpy))

123668.57102607541